In [3]:
import pandas as pd
import os
from pydantic import BaseModel
from ollama import generate
import platform
import datetime
from dataset_processing import create_list_of_commands

operating_system = platform.system()
if operating_system == 'Windows':
    raise Exception("not used on Windows yet")
elif operating_system == 'Linux':
    base_directory = "/home/chris/gdrive/work/annabell"
elif operating_system == 'Darwin':  #macOS
    base_directory = "/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/"
else:
    raise Exception("unsupported OS")

number_of_pretraining_samples = 5

timestamp = datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")
data_directory = os.path.join(base_directory, "experiments/data")
dataset_filename = "response_formatted_20250924_174653.jsonl"
dataset_filepath = os.path.join(data_directory, dataset_filename)
base_prompt_directory = os.path.join(data_directory, "prompts")
base_prompt_filename = "prompt_for_annabell_pretraining_command_generation.txt"
base_prompt_filepath = os.path.join(base_prompt_directory, base_prompt_filename)
nyc_squad_df = pd.read_json(dataset_filepath, lines=True)
#drop all  the columns except for id response_question, response_answer, response_declarative_sentence
nyc_squad_df_trimmed = nyc_squad_df[
    ["id", "response_question_formatted", "response_answer_formatted", "response_declarative_sentence_formatted"]]
#take a random sample of 200 rows from the dataframe
nyc_squad_sample_df = nyc_squad_df_trimmed.sample(n=number_of_pretraining_samples, random_state=42).reset_index(
    drop=True)
#create a list of prompt_inputs in JSON lines format
prompt_inputs = []
for index, row in nyc_squad_sample_df.iterrows():
    json_line = row.to_json()
    prompt_inputs.append(json_line)

with open(base_prompt_filepath, "r") as base_prompt_file:
    base_prompt = base_prompt_file.read()

In [4]:
class PretrainingCommand(BaseModel):
    id: str
    commands: list[str]


def generated_text_from_prompt(the_model_string, the_prompt, the_format=None, the_options=None):
    generated_text = generate(model=the_model_string, prompt=the_prompt, format=the_format, options=the_options)
    return generated_text.response


def generated_json_from_prompt(the_model_string, the_prompt, the_options=None):
    the_response = generated_text_from_prompt(
        the_model_string=the_model_string,
        the_prompt=the_prompt,
        the_format=PretrainingCommand.model_json_schema(),
        the_options=the_options)
    return PretrainingCommand.model_validate_json(the_response)

In [5]:
model_string = "mistral-nemo"
results = []
for prompt_input_sample in prompt_inputs:
    prompt = base_prompt + prompt_input_sample
    result = generated_text_from_prompt(model_string, prompt)
    results.append(result)
results

['{"id":"56cfdde6234ae51400d9bfa8","training_commands": ["10 leader terrorists of Al Qaeda were involved with the 9/11 attacks directly that day", "? how many leader terrorists of Al Qaeda were involved with the 9/11 attacks directly that day", ".wg leader", ".wg terrorists", ".wg involved", ".wg Al Qaeda", ".wg 9/11 attacks", ".ph 10 leader terrorists of Al Qaeda were involved with the 9/11 attacks directly that day", ".wg 10", ".rw"]}',
 '{"id":"56cf578daab44d1400b89089","training_commands": ["the Halloween Parade take -s place in the Greenwich Village neighborhood", "? in what neighborhood does the Halloween Parade take place", ".wg neighborhood", ".wg Greenwich Village", ".wg Halloween Parade", ".ph the Halloween Parade take -s place in the Greenwich Village neighborhood", ".wg Greenwich Village", ".rw"]}',
 '{"id":"56cf1a05aab44d1400b88d7f","training_commands": ["there are 50 buildings in New York City that are over 200m high", "? how many buildings in New York City are over 200m 

In [6]:
#save the results as a JSON lines file
with open(os.path.join(data_directory, "pretraining_commands_responses.jsonl"), "w") as results_file:
    for result in results:
        results_file.write(result + "\n")
#load the file back in as a dataframe
results_df = pd.read_json(os.path.join(data_directory, "pretraining_commands_responses.jsonl"), lines=True)
results_df.head()
#append the training_commands column to the original dataframe using the id column to match
final_df = pd.merge(nyc_squad_df_trimmed, results_df, on="id", how="left")
final_df = pd.merge(nyc_squad_df_trimmed, results_df, on="id", how="left")
final_df["training_commands"] = final_df["training_commands"].apply(lambda x: x if isinstance(x, list) else [])

,id,response_question_formatted,response_answer_formatted,response_declarative_sentence_formatted,training_commands
0,56ce304daab44d1400b8850e,? what city in the United States has the high ...,New York,the city in the United States with the high -e...,[]
1,56ce304daab44d1400b8850f,? in what city is the United Nations base -d,New York,the United Nations is base -d in New York,[]
2,56ce304daab44d1400b88510,? what city has been call -ed the culture -al ...,New York,New York has been call -ed the culture -al cap...,[]
3,56ce304daab44d1400b88511,? what American city welcome -s the large -st ...,New York,New York is the American city that welcome -s ...,[]
4,56cf5d41aab44d1400b89130,? the major gateway for immigrate -ion has bee...,New York City,the major gateway for immigrate -ion has been ...,[]
...,...,...,...,...,...
812,56d1218c17492d1400aaba1f,? how much money in cent -s does New York City...,83,New York City receive -s 83 cent -s for every ...,[]
813,56d1218c17492d1400aaba20,? how much more money does the city give to th...,11 billion,the city give -s 11 billion more money to the...,[]
814,56d1218c17492d1400aaba21,? each year how much more money does New York ...,11 point 4 billion,New York City give -s 11 point 4 billion more...,[]
815,56d121d817492d1400aaba2d,? what is the new name of the Sister City Prog...,New York City Global Partners,the new name of the Sister City Program of the...,[]


##prepare the text using nltk methods

In [21]:
#add a new column to the dataframe with the created list of commands
final_df["created_commands"] = final_df.apply(create_list_of_commands, axis=1)

#save the final dataframe as a JSON lines file
final_df.to_json(os.path.join(data_directory, "nyc_squad_with_pretraining_commands.jsonl"), orient="records",
                 lines=True)

In [7]:
#write a training file from the LLM generated commands
with open(os.path.join(data_directory, "nyc_squad_pretraining_commands_automatic.txt"), "w") as commands_file:
    for index, row in final_df.iterrows():
        commands = row["training_commands"]
        for command in commands:
            commands_file.write(command + "\n")
with open(os.path.join(data_directory, "nyc_squad_pretraining_commands_automatic.txt"), "r") as commands_file:
    lines = commands_file.readlines()
    print(f"Number of commands: {len(lines)}")
    print("First 5 commands:")
    for line in lines[:100]:
        print(line.strip())

Number of commands: 45
First 5 commands:
10 leader terrorists of Al Qaeda were involved with the 9/11 attacks directly that day
? how many leader terrorists of Al Qaeda were involved with the 9/11 attacks directly that day
.wg leader
.wg terrorists
.wg involved
.wg Al Qaeda
.wg 9/11 attacks
.ph 10 leader terrorists of Al Qaeda were involved with the 9/11 attacks directly that day
.wg 10
.rw
there are 50 buildings in New York City that are over 200m high
? how many buildings in New York City are over 200m high
.wg buildings
.wg New York City
.wg over
.wg 200m
.ph there are 50 buildings in New York City that are over 200m high
.wg 50
.rw
the Halloween Parade take -s place in the Greenwich Village neighborhood
? in what neighborhood does the Halloween Parade take place
.wg neighborhood
.wg Greenwich Village
.wg Halloween Parade
.ph the Halloween Parade take -s place in the Greenwich Village neighborhood
.wg Greenwich Village
.rw
475000 uninsure-d New Yorker-s take advantage of HHC
? how m

In [23]:
# write a training file from the programmatically created commands
with open(os.path.join(data_directory, "nyc_squad_pretraining_commands_programmatic.txt"), "w") as commands_file:
    for index, row in final_df.iterrows():
        commands = row["created_commands"]
        for command in commands:
            commands_file.write(command + "\n")
with open(os.path.join(data_directory, "nyc_squad_pretraining_commands_programmatic.txt"), "r") as commands_file:
    lines = commands_file.readlines()
    print(f"Number of commands: {len(lines)}")
    print("First 100 commands:")
    for line in lines[:100]:
        print(line.strip())

Number of commands: 10956
First 100 commands:
# ID: 56ce304daab44d1400b8850e
the city in the United States with the high -est populate -ion is New York
? what city in the United States has the high -est populate -ion
.wg city
.wg United
.wg States
.wg high
.wg -est
.wg populate
.wg -ion
.ph the city in the United States with the high -est populate -ion is New York
.wg New York
.rw
# ID: 56ce304daab44d1400b8850f
the United Nations is base -d in New York
? in what city is the United Nations base -d
.wg United
.wg Nations
.wg base
.wg -d
.ph the United Nations is base -d in New York
.wg New York
.rw
# ID: 56ce304daab44d1400b88510
New York has been call -ed the culture -al capital of the world
? what city has been call -ed the culture -al capital of the world
.wg call
.wg -ed
.wg culture
.wg -al
.wg capital
.wg world
.ph New York has been call -ed the culture -al capital of the world
.wg New York
.rw
# ID: 56ce304daab44d1400b88511
New York is the American city that welcome -s the large -st

In [22]:
final_df

,id,response_question_formatted,response_answer_formatted,response_declarative_sentence_formatted,training_commands,created_commands
0,56ce304daab44d1400b8850e,? what city in the United States has the high ...,New York,the city in the United States with the high -e...,[],"[# ID: 56ce304daab44d1400b8850e, the city in t..."
1,56ce304daab44d1400b8850f,? in what city is the United Nations base -d,New York,the United Nations is base -d in New York,[],"[# ID: 56ce304daab44d1400b8850f, the United Na..."
2,56ce304daab44d1400b88510,? what city has been call -ed the culture -al ...,New York,New York has been call -ed the culture -al cap...,[],"[# ID: 56ce304daab44d1400b88510, New York has ..."
3,56ce304daab44d1400b88511,? what American city welcome -s the large -st ...,New York,New York is the American city that welcome -s ...,[],"[# ID: 56ce304daab44d1400b88511, New York is t..."
4,56cf5d41aab44d1400b89130,? the major gateway for immigrate -ion has bee...,New York City,the major gateway for immigrate -ion has been ...,[],"[# ID: 56cf5d41aab44d1400b89130, the major gat..."
...,...,...,...,...,...,...
812,56d1218c17492d1400aaba1f,? how much money in cent -s does New York City...,83,New York City receive -s 83 cent -s for every ...,[],"[# ID: 56d1218c17492d1400aaba1f, New York City..."
813,56d1218c17492d1400aaba20,? how much more money does the city give to th...,11 billion,the city give -s 11 billion more money to the...,[],"[# ID: 56d1218c17492d1400aaba20, the city give..."
814,56d1218c17492d1400aaba21,? each year how much more money does New York ...,11 point 4 billion,New York City give -s 11 point 4 billion more...,[],"[# ID: 56d1218c17492d1400aaba21, New York City..."
815,56d121d817492d1400aaba2d,? what is the new name of the Sister City Prog...,New York City Global Partners,the new name of the Sister City Program of the...,[],"[# ID: 56d121d817492d1400aaba2d, the new name ..."
